# Práctico

El trabajo práctico de la materia consiste en el análisis de un conjunto de datos extraído de Twitter. La idea es emplear los conceptos de grafos vistos en clase sobre un caso real de actualidad.

## Dataset

El dataset consiste en un conjunto de hilos de tweets, con un total de ~150000 tweets, extraídos entre Enero y Marzo de 2021. La temática de los mismos está referida a la vacunación contra el covid-19 en Argentina.

Pueden descargar el dataset del siguiente [link](https://drive.google.com/file/d/1X_qKsE8muAnom2tDX4sLlmBAO0Ikfe_G/view?usp=sharing).

### Campos

- **created_at:** Fecha del tweet
- **id_str:** ID del tweet
- **full_text:** Contenido del tweet
- **in_reply_to_status_id:** ID del tweet inmediatamente anterior en el hilo
- **in_reply_to_user_id:** Autor del tweet inmediatamente anterior en el hilo
- **user.id:** Autor del tweet
- **user_retweeters:** Lista de ID de usuarios que retweetearon el tweet
- **sentiment:** Etiquetado manual que indica el sentimiento o intención del tweet con respecto al tweet anterior en el hilo

## Configuración inicial

In [1]:
import pandas as pd
from pathlib import Path
import networkx as nx
import igraph as ig

import matplotlib.pyplot as plt
import numpy as np
from time import time

## Descargar el csv con los datos en este directorio
DATA_DIR = Path('./data')
INPUT_FILE = DATA_DIR / 'vacunas.csv'

## Creamos el directorio en caso de que no exista
DATA_DIR.mkdir(parents=True, exist_ok=True)

### Cargamos el dataset

In [2]:
dtypes = {
    'id_str': str,
    'full_text': str,
    'in_reply_to_status_id': str,
    'in_reply_to_user_id': str,
    'user.id': str
}
df = pd.read_csv(INPUT_FILE, dtype=dtypes).dropna(subset=['user_retweeters'])
df['user_retweeters'] = df['user_retweeters'].apply(lambda x: [str(elem) for elem in eval(x)])
print(df.shape)
df.head()

(155123, 8)


,created_at,id_str,full_text,in_reply_to_status_id,in_reply_to_user_id,user.id,user_retweeters,sentiment
0,Sat Feb 20 03:09:10 +0000 2021,1362962469749153792,Seguimos esperando el comunicado de @norabar r...,NaN,NaN,2737379453,"[2258074658, 159909978, 105301854, 290671142, ...",NaN
1,Sat Feb 20 03:19:59 +0000 2021,1362965193509265417,@Clon_43 @norabar Nora estaba indignada porque...,1362962469749153792,2737379453,32718111,[],NaN
2,Mon Feb 22 23:55:08 +0000 2021,1364000806740111363,"Bueno, Alberto dijo Salud o Economía. La salud...",NaN,NaN,252168075,"[1238117630696972289, 37232479, 12792246571247...",NaN
3,Tue Feb 23 00:09:14 +0000 2021,1364004354374696963,@spitta1969 Tuit del mes Spitta,1364000806740111363,252168075,1156346340802224128,[],NaN
4,Tue Feb 23 00:00:17 +0000 2021,1364002100364128260,@spitta1969 Estas onfire,1364000806740111363,252168075,153663816,[],NaN


### Observamos algunos ejemplos

In [3]:
idx = 0
print('Texto:', df.full_text.values[idx])
print('Retweets:', len(df.user_retweeters.values[idx]))

Texto: Seguimos esperando el comunicado de @norabar repudiando la situación respecto del gobierno y el tema vacunas. Seamos pacientes que con esto de la pandemia anda con mucho "laburo".
Retweets: 9


In [4]:
idx = 376
print('Text:', df.full_text.values[idx])
print('Retweets:', len(df.user_retweeters.values[idx]))

Text: Todo lo que hay que entender sobre la decisión –o no– de poner más vacunas en más brazos (por ejemplo, usar las 1º dosis en muchos y si es necesario retrasar la 2º) está en esta excelente nota de Nora Bär. https://t.co/A0I03DyxgO
Retweets: 48


### Calculamos la cantidad de hilos

In [5]:
roots = df[df['in_reply_to_user_id'].isna()]
roots.shape

(3174, 8)

## Actividades

### Primera parte

#### **1. Construcción del grafo** 

Construir el **grafo de retweets**, definido de la siguiente manera:

- Tipo de grafo: Dirigido
- Nodos: ID de los usuarios
- Enlaces: (Usuario A) ---> (Usuario B) si B retweeteó algún tweet de A

Con estos datos, el grafo debería tener alrededor de 40000 nodos y 90000 enlaces.

Considerar la versión no dirigida del grafo y estudiar su conectividad. Si existe una única "componente gigante", realizar el resto de las actividades sobre ella, en lugar de sobre el grafo completo.

Calcular las siguientes métricas globales del grafo:

- Grado medio
- Asortatividad
- Transitividad
- Coeficiente de clustering de Watts-Strogatz

**Opcional:** Comparar las métricas calculadas anteriormente con las de un grafo aleatorio con la misma distribución de grado. Pueden utilizar para ello este [método](https://networkx.org/documentation/stable/reference/generated/networkx.generators.degree_seq.configuration_model.html?highlight=configuration#networkx.generators.degree_seq.configuration_model). Con esto en mente, comentar si los valores obtenidos anteriormente difieren significativamente del caso aleatorio.




In [6]:
df

,created_at,id_str,full_text,in_reply_to_status_id,in_reply_to_user_id,user.id,user_retweeters,sentiment
0,Sat Feb 20 03:09:10 +0000 2021,1362962469749153792,Seguimos esperando el comunicado de @norabar r...,NaN,NaN,2737379453,"[2258074658, 159909978, 105301854, 290671142, ...",NaN
1,Sat Feb 20 03:19:59 +0000 2021,1362965193509265417,@Clon_43 @norabar Nora estaba indignada porque...,1362962469749153792,2737379453,32718111,[],NaN
2,Mon Feb 22 23:55:08 +0000 2021,1364000806740111363,"Bueno, Alberto dijo Salud o Economía. La salud...",NaN,NaN,252168075,"[1238117630696972289, 37232479, 12792246571247...",NaN
3,Tue Feb 23 00:09:14 +0000 2021,1364004354374696963,@spitta1969 Tuit del mes Spitta,1364000806740111363,252168075,1156346340802224128,[],NaN
4,Tue Feb 23 00:00:17 +0000 2021,1364002100364128260,@spitta1969 Estas onfire,1364000806740111363,252168075,153663816,[],NaN
...,...,...,...,...,...,...,...,...
163174,Tue Jan 12 23:24:10 +0000 2021,1349135109010677767,@Die_IsCast @norabar @enzosebastin221 Retuitea...,1349122147587940353,2955376486,176835482,[],NaN
163175,Wed Jan 13 01:56:56 +0000 2021,1349173553833381888,"@todosflotan @norabar @enzosebastin221 Sí, sí,...",1349135109010677767,176835482,2955376486,[],NaN
163176,Wed Jan 13 02:02:07 +0000 2021,1349174860702953474,@Die_IsCast @norabar @enzosebastin221 seguí co...,1349173553833381888,2955376486,176835482,[],NaN
163177,Tue Jan 12 15:07:16 +0000 2021,1349010059712491522,"@mirtapsp @todosflotan @norabar Nora, trabaja ...",1349002034968719360,138734328,1219302815916527618,[],NaN


pos = nx.spring_layout(G) #specify layout for visual

In [92]:
G = nx.Graph()


In [93]:
edges = df.apply( lambda row : [( row['user.id'] , v ) for v in row['user_retweeters']] ,  axis=1 )


In [94]:
for edge in edges:
    G.add_edges_from(edge)

In [95]:
print(nx.info(G))


Name: 
Type: Graph
Number of nodes: 39800
Number of edges: 93404
Average degree:   4.6937


## Grado medio

In [11]:
deg_seq = np.array([k for v, k in G.degree()])

grado_medio = sum(deg_seq) / len(deg_seq)
grado_medio

4.7081407035175875

## Asortatividad

In [12]:
assortativity = nx.degree_pearson_correlation_coefficient(G)
assortativity

-0.25828280140489684

## Transitividad 

In [13]:
transitivity = nx.transitivity(G)
transitivity

0.00033160843475072696

## Coeficiente de clustering de Watts-Strogatz

In [14]:
avg_clustering = nx.average_clustering(G)
avg_clustering

0.06073576789848228

#### **2. Centralidad**

Calcular 5 métricas de centralidad de nodos. Graficar la distribución de cada una de ellas ¿Existe alguna correlación entre las distintas centralidades? 

Hacer un ranking con los 10 nodos más centrales para cada métrica. ¿Hay coincidencia entre los rankings?. ¿Qué características tienen los usuarios más centrales y sus respectivos tweets?

**Opcional:** Determinar si existe alguna correlación entre la centralidad de un nodo y su actividad en red social. Es decir, evaluar si los usuarios que más escriben son los más centrales o no.



In [15]:
# Netoworkx no performa bien calculando algunas medidas de centralidad, es por ello que se prueba con igraph

g_ig = ig.Graph.TupleList(G.edges())
print(g_ig.summary())

IGRAPH UN-- 39800 93692 -- 
+ attr: name (v)


In [16]:
df_metrics = g_ig.get_vertex_dataframe()


In [17]:
btw = g_ig.betweenness()

In [18]:
df_metrics['btw'] = btw


In [19]:
closeness = g_ig.closeness()
df_metrics['closeness']  = closeness


In [20]:
degree = g_ig.degree()

In [21]:
df_metrics['degree'] = degree


In [22]:
eigenvector_centrality = g_ig.eigenvector_centrality()

In [23]:
df_metrics['eigenvector_centrality'] = eigenvector_centrality


In [24]:
page_rank = g_ig.pagerank()

In [25]:
df_metrics['page_rank'] = page_rank


In [26]:
har_centr = g_ig.harmonic_centrality()

In [27]:
df_metrics['har_centr'] = har_centr


In [28]:
df_metrics


,name,btw,closeness,degree,eigenvector_centrality,page_rank,har_centr
vertex ID,,,,,,,
0,2737379453,78110.382425,0.244898,9,0.000956,0.000038,0.250772
1,2258074658,129537.220362,0.286305,19,0.012419,0.000069,0.307703
2,159909978,0.000000,0.196722,1,0.000009,0.000007,0.198210
3,105301854,31182.763024,0.304398,10,0.013902,0.000036,0.325685
4,290671142,290056.063418,0.308014,15,0.014944,0.000056,0.329096
...,...,...,...,...,...,...,...
39795,2765081259,0.000000,1.000000,1,0.000000,0.000025,0.000025
39796,339254354,42342.871206,0.276895,6,0.000429,0.000031,0.281808
39797,346605471,0.000000,0.216851,1,0.000004,0.000008,0.217565


In [30]:
def get_top_usrs_by_metrics(columns, top_n , df ):
   
    data = {}
    for col in columns:
        top_families = df.nlargest(top_n, columns=[col])[col].index
        data[col] = top_families
    return pd.DataFrame(data)

In [31]:
columns = ['btw', 'closeness', 'degree', 'eigenvector_centrality', 'page_rank', 'har_centr']

df_top_by_metrics = get_top_usrs_by_metrics(columns, 10 , df_metrics)

In [32]:
df_top_by_metrics

,btw,closeness,degree,eigenvector_centrality,page_rank,har_centr
0,20,27466,20,20,20,20
1,15271,27479,15271,15271,15271,15271
2,7996,27480,7996,7996,7996,7996
3,24746,28063,13554,21518,24746,13554
4,21518,28064,24746,13554,13554,4339
5,13554,28176,12756,12756,12756,21518
6,12756,28377,21518,24746,23095,6667
7,23095,28378,23095,13442,21518,7725
8,13442,28602,13442,23095,13442,2643
9,6762,28603,14291,446,14291,12756


 ¿Hay coincidencia entre los rankings?. ¿Qué características tienen los usuarios más centrales y sus respectivos tweets?

## HACER

In [34]:
df_metrics.iloc[[20,15271,7996,13554,24746]]


,name,btw,closeness,degree,eigenvector_centrality,page_rank,har_centr
vertex ID,,,,,,,
20,252168075,2.721111e+08,0.392229,8366,1.000000,0.040935,0.478310
15271,130979339,1.797415e+08,0.398115,5558,0.347800,0.033172,0.457576
7996,73102744,1.255262e+08,0.371290,5367,0.311946,0.030850,0.434693
13554,367933714,9.243807e+07,0.382177,3852,0.237615,0.021960,0.431454
24746,593189095,1.138296e+08,0.349051,3835,0.119383,0.029132,0.393269


In [76]:
central_users =df[df['user.id'].isin(['252168075','130979339','73102744','367933714','593189095'])]

In [42]:
central_users.columns

Index(['created_at', 'id_str', 'full_text', 'in_reply_to_status_id',
       'in_reply_to_user_id', 'user.id', 'user_retweeters', 'sentiment'],
      dtype='object')

In [77]:
central_users.describe()

,created_at,id_str,full_text,in_reply_to_status_id,in_reply_to_user_id,user.id,user_retweeters,sentiment
count,6394,6394,6394,4905,4910,6394,6394,17
unique,6267,6392,6368,4879,1990,5,3703,3
top,Wed Mar 10 22:51:37 +0000 2021,1366950420350988294,@juliaquetglas2 ❤️,1356774782730203136,252168075,73102744,[],Comentario
freq,4,2,6,3,703,2195,2457,13


In [51]:
central_users.fillna(0,inplace=True)

C:\Users\Magui\anaconda3\lib\site-packages\pandas\core\frame.py:4462: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


In [53]:
central_users.groupby(by=['user.id','sentiment']).id_str.count()

user.id    sentiment 
130979339  0             1061
252168075  0             1948
           Apoyo            1
           Comentario       3
           Pregunta         1
367933714  0              776
           Comentario       6
593189095  0              402
           Comentario       1
73102744   0             2190
           Apoyo            2
           Comentario       3
Name: id_str, dtype: int64


Son usuarios con muchos tweets, de hecho el 73102744 que está en tercer lugar en casi todas las medidas de centralidad es el que más tuits publicó. No se puede decir nada sobre sus sentimientos pues no están todos tageados.

La gran mayoría de sus tuits son respuestas a otros usuarios

In [57]:
df1 = df.fillna(0).groupby(by=['user.id']).id_str.count()

In [71]:
df1.sort_values( ascending=False).head(15)

user.id
73102744               2195
252168075              1953
130979339              1061
2687724840              893
367933714               782
1077176953              410
593189095               403
312708081               290
3015178264              289
1273799426348331012     254
931564592328781824      243
956981248299282432      220
144929758               205
35680704                171
4343677427              167
Name: id_str, dtype: int64

In [78]:
df[df['user.id'].isin(['73102744'])].head(10)

,created_at,id_str,full_text,in_reply_to_status_id,in_reply_to_user_id,user.id,user_retweeters,sentiment
39,Sun Feb 28 23:51:37 +0000 2021,1366174247425556483,Argentina tiene hoy casi 18 muertos por millón...,NaN,NaN,73102744,"[178105343, 2644056245, 4196856351, 215547565,...",NaN
107,Mon Mar 01 05:09:45 +0000 2021,1366254308426207232,@arroadri Inevitable a menos que se aceleren l...,1366218359730241543,171893542,73102744,[1083825689848135680],NaN
111,Mon Mar 01 00:53:09 +0000 2021,1366189731235643394,"@jadagui un rato, me cayeron los violentos",1366184037543460865,139058257,73102744,[],NaN
114,Mon Mar 01 12:17:42 +0000 2021,1366362007063117827,@PTorricos Precisamente y cuidadosamente dije ...,1366357963099750404,1233442742312865792,73102744,[],NaN
116,Mon Mar 01 02:43:39 +0000 2021,1366217542889598977,@Horacio86925173 Puse candado para evitar a lo...,1366185744574930944,1351278834147069953,73102744,[],NaN
119,Mon Mar 01 00:05:53 +0000 2021,1366177838932631554,@cacerolapop 🥰,1366176472357101571,4497008602,73102744,[],NaN
120,Mon Mar 01 03:32:38 +0000 2021,1366229867675664384,@musa_nancy ❤️,1366229748498718720,1873909063,73102744,[],NaN
122,Mon Mar 01 03:49:05 +0000 2021,1366234007118225415,@musa_nancy Abrazo!,1366230372762218501,1873909063,73102744,[],NaN
124,Mon Mar 01 00:51:33 +0000 2021,1366189329543032840,"@vidal_aleta Se ve que no leiste nada, no te a...",1366186725341593601,984448926395793408,73102744,[],NaN
128,Mon Mar 01 00:57:15 +0000 2021,1366190763625873415,@vidal_aleta No leíste el hilo completo y diji...,1366190098119860226,984448926395793408,73102744,[],NaN


#### **3. Comunidades**

Utilizar el algoritmo de Louvain con el parámetro "resolución" igual a 1. Caracterizar las comunidades halladas (cantidad, distribución de tamaños). Utilizar la modularidad y otras dos métricas a elección para evaluar la calidad de la partición encontrada. 

Variar el parámetro "resolución" y observar cómo cambia la distribución de comunidades encontradas. ¿Existe algún valor para el cual se identifiquen dos grandes comunidades?

Elegir otro algoritmo de detección de comunidades y comparar los resultados con los obtenidos anteriormente.

**Opcional:** Correr el algoritmo de Louvain con distintas semillas aleatorias. Utilizar alguna métrica de comparación externa entre las particiones obtenidas para determinar en qué medida depende el algoritmo de la condición inicial.

In [87]:
import networkit as nk


In [96]:
g_nk = nk.nxadapter.nx2nk(G)


In [97]:
plmCommunities_1 = nk.community.detectCommunities(g_nk, algo=nk.community.PLM(G = g_nk, refine = True , gamma = 1))


Communities detected in 0.08295 [s]
solution properties:
-------------------  -----------
# communities         365
min community size      1
max community size   7574
avg. community size   109.041
modularity              0.585817
-------------------  -----------


In [98]:
def get_communidades(communities):

    
    values = []

    for index in range(communities.numberOfSubsets()):


        values.append([communities.getMembers(index) , len(communities.getMembers(index)) ])
        df = pd.DataFrame(values, columns = [ 'miembros' , 'cantidad'])

    return df

In [99]:
plmCommunities_05 = nk.community.detectCommunities(g_nk, algo=nk.community.PLM(G = g_nk, refine = True , gamma = 0.3))



Communities detected in 0.06296 [s]
solution properties:
-------------------  ------------
# communities          339
min community size       1
max community size   22668
avg. community size    117.404
modularity               0.504088
-------------------  ------------


In [100]:
df = get_communidades(plmCommunities_05)
sorted_df = df.sort_values(axis=0, by=['cantidad'] , ascending=False)
sorted_df[:10]

,miembros,cantidad
1,"{87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 9...",22668
0,"{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",15652
23,"{27381, 7208, 20191, 20192, 20193, 20198, 2019...",189
34,"{9318, 9319, 9320, 9321, 9322, 29978, 29979, 2...",106
4,"{30748, 23583, 23584, 23585, 1589, 1590, 14929...",97
17,"{26633, 26634, 26635, 26636, 26637, 26638, 266...",85
8,"{33793, 33794, 33795, 33796, 33797, 33798, 337...",72
40,"{34048, 34049, 34050, 34051, 34052, 34053, 340...",50
12,"{3968, 3969, 3970, 3971, 3972, 3973, 3974, 397...",45
261,"{32658, 32659, 32660, 32661, 32662, 32663, 326...",32


## Segunda parte

### **4. Extracción de etiquetas**

En el archivo [etiquetas.csv](https://drive.google.com/file/d/1LWY3VoIRt0xKwEbbtMXYePOGZvgPsQh-/view?usp=sharing) están las etiquetas para un pequeño subconjunto de nodos. Podemos interpretar el valor de la etiqueta como la pertenencia a una determinada clase, donde los usuarios de una misma clase en general tienden a expresar apoyo entre sí.

- Determinar quiénes son los usuarios referentes de cada clase (utilizar alguna medida de centralidad calculada sobre el grafo de retweets).
- Utiliando los resultados del práctico anterior, determinar si los usuarios de cada clase forman parte de distintas comunidades.

**Opcional:** Reconstruir el archivo "etiquetas.csv". Para eso, hacer lo siguiente

- Construir un grafo en donde los nodos sean usuarios, y donde los enlaces unan dos nodos si entre ellos hubo más respuestas de apoyo que de oposición.
- Extraer las dos componentes más grandes del grafo. Esos serán nuestros nodos etiquetados.

### **5. Embedding de nodos**

- Generar un embedding del grafo de retweets utilizando el algoritmo `word2vec`.
- Reducir a 2 la dimensionalidad del embedding utilizando [PCA](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.htmlhttps://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html) y [t-SNE](https://scikit-learn.org/stable/modules/generated/sklearn.manifold.TSNE.htmlhttps://scikit-learn.org/stable/modules/generated/sklearn.manifold.TSNE.html).
- Graficar los embeddings correspondientes a los datos etiquetados. ¿Es posible diferenciar unos de otros?

**Opcional:** Graficar además los embeddings de los nodos que forman parte de las comunidades asociadas a cada clase. Determinar si el embedding permite distinguir cada comunidad.

### **Opcional: 6. Redes neuronales de grafos**

El archivo [word_vectors.csv](https://drive.google.com/file/d/1aoxugyMktKb0NQ8Pf3bdhKvh8BAj7YZz/view?usp=sharing) contiene un embedding de 300 dimensiones para cada tweet, otenido utilizando un modelo preentrenado de [FastText](https://fasttext.cc/). Construir una matriz de features para los nodos tomando, para cada usuario, el promedio de los vectores correspondientes a los tweets que escribió. Utilizando estos features, y tomando como ejemplos etiquetados los usuarios de "etiquetas.csv" entrenar una red neuronal de grafos para realizar una clasificación binaria sobre el resto de los nodos. Pueden utilizar como base el siguiente modelo:

In [ ]:
class GCN(torch.nn.Module):
    def __init__(self):
        super(GCN, self).__init__()
        torch.manual_seed(1234)
        self.conv1 = GCNConv(dataset.num_features, 4)
        self.conv2 = GCNConv(4, 4)
        self.conv3 = GCNConv(4, 2)
        self.classifier = Linear(2, dataset.num_classes)

    def forward(self, x, edge_index):
        h = self.conv1(x, edge_index)
        h = h.tanh()
        h = self.conv2(h, edge_index)
        h = h.tanh()
        h = self.conv3(h, edge_index)
        h = h.tanh()  # Embedding final
        
        # Aplicamos un clasificador lineal sobre el embedding
        out = self.classifier(h)

        return out, h

**Observación:** para alimentar la red neuronal, es necesario construir un objeto de la clase `Dataset` de PyTorch-Geometric. Una forma de hacer eso es la siguiente

In [ ]:
from torch_geometric.data import InMemoryDataset, Data

## Reemplazar por el grafo correspondiente
g = nx.Graph()

## Etiquetas. Reemplazar por las clases del archivo 'etiquetas.csv'.
## Asignar la clase '2' a los ejemplos no etiquetados
labels = [1, 0, 2, ..., 1]

## True si el ejemplo está etiquetado (clases 0 y 1)
train_idx = [True, True, False, ..., True]

## Matriz de features (word vectors)
features = ...

adj = nx.to_scipy_sparse_matrix(g).tocoo()
row = torch.from_numpy(adj.row.astype(np.int64)).to(torch.long)
col = torch.from_numpy(adj.col.astype(np.int64)).to(torch.long)
edge_index = torch.stack([row, col], dim=0)


class TwitterDataset(InMemoryDataset):
    def __init__(self, transform=None):
        super(TwitterDataset, self).__init__('.', transform, None, None)

        data = Data(edge_index=edge_index)
        
        data.num_nodes = g.number_of_nodes()
        
        # Features 
        data.x = torch.from_numpy(features).type(torch.float32)
        
        # Etiquetas
        y = torch.from_numpy(labels).type(torch.long)
        data.y = y.clone().detach()
        
        data.num_classes = 2
        
        n_nodes = g.number_of_nodes()
        
        # create train and test masks for data
        train_mask = torch.zeros(n_nodes, dtype=torch.bool)
        train_mask[train_idx] = True
        data['train_mask'] = train_mask

        self.data, self.slices = self.collate([data])

    def _download(self):
        return

    def _process(self):
        return

    def __repr__(self):
        return '{}()'.format(self.__class__.__name__)